In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({0:1,1:0,2:0},inplace=True)

In [29]:
# Divide data in half for tuning, to reduce leakage when using split/repeat cross-validation
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

In [18]:
y.value_counts()

0.0    429
1.0    353
Name: CDGLOBAL, dtype: int64

# RF Classifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 8.4 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.955093,0.872727,0.918112,0.913910,0.872727,0.955093,0.918112,0.913910,0.872727,0.955093,0.918112,0.913910
std,0.033891,0.050498,0.017240,0.018792,0.050498,0.033891,0.017240,0.018792,0.050498,0.033891,0.017240,0.018792
min,0.907407,0.818182,0.887755,0.884470,0.818182,0.907407,0.887755,0.884470,0.818182,0.907407,0.887755,0.884470
25%,0.916667,0.838068,0.906888,0.898674,0.838068,0.916667,0.906888,0.898674,0.838068,0.916667,0.906888,0.898674
50%,0.958333,0.852273,0.915816,0.913826,0.852273,0.958333,0.915816,0.913826,0.852273,0.958333,0.915816,0.913826
75%,0.990741,0.911932,0.929847,0.926399,0.911932,0.990741,0.929847,0.926399,0.911932,0.990741,0.929847,0.926399
max,1.000000,0.965909,0.954082,0.955177,0.965909,1.000000,0.954082,0.955177,0.965909,1.000000,0.954082,0.955177


## Random Search

In [20]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [21]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 10.6min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=400, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [22]:
rf_search_rand.best_params_

{'n_estimators': 1400,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 0.3,
 'max_depth': 80,
 'bootstrap': False}

In [23]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.930041,0.852273,0.895125,0.891157,0.852273,0.930041,0.895125,0.891157,0.852273,0.930041,0.895125,0.891157
std,0.023657,0.035483,0.013998,0.014984,0.035483,0.023657,0.013998,0.014984,0.035483,0.023657,0.013998,0.014984
min,0.888889,0.806818,0.877551,0.876263,0.806818,0.888889,0.877551,0.876263,0.806818,0.888889,0.877551,0.876263
25%,0.907407,0.806818,0.887755,0.880261,0.806818,0.907407,0.887755,0.880261,0.806818,0.907407,0.887755,0.880261
50%,0.935185,0.863636,0.887755,0.885522,0.863636,0.935185,0.887755,0.885522,0.863636,0.935185,0.887755,0.885522
75%,0.953704,0.886364,0.913265,0.910774,0.886364,0.953704,0.913265,0.910774,0.886364,0.953704,0.913265,0.910774
max,0.953704,0.886364,0.913265,0.910774,0.886364,0.953704,0.913265,0.910774,0.886364,0.953704,0.913265,0.910774


## Grid Search

In [24]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [1300, 1400, 1500],
               'max_features': [.2, .3, .4],
               'max_depth': [70, 80, 90],
               'min_samples_split': [1, 2, 3],
               'min_samples_leaf': [1, 2, 3],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  8.8min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_features=0.5,
                                              random_state=555),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True], 'max_depth': [70, 80, 90],
                         'max_features': [0.2, 0.3, 0.4],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3],
                         'n_estimators': [1300, 1400, 1500]},
             verbose=2)

In [25]:
rf_search_grid.best_params_

{'bootstrap': False,
 'max_depth': 70,
 'max_features': 0.2,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1500}

In [26]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.929012,0.852273,0.894558,0.890643,0.852273,0.929012,0.894558,0.890643,0.852273,0.929012,0.894558,0.890643
std,0.017930,0.038953,0.014431,0.016199,0.038953,0.017930,0.014431,0.016199,0.038953,0.017930,0.014431,0.016199
min,0.907407,0.795455,0.882653,0.874579,0.795455,0.907407,0.882653,0.874579,0.795455,0.907407,0.882653,0.874579
25%,0.907407,0.806818,0.882653,0.875631,0.806818,0.907407,0.882653,0.875631,0.806818,0.907407,0.882653,0.875631
50%,0.935185,0.863636,0.887755,0.885522,0.863636,0.935185,0.887755,0.885522,0.863636,0.935185,0.887755,0.885522
75%,0.944444,0.886364,0.908163,0.906145,0.886364,0.944444,0.908163,0.906145,0.886364,0.944444,0.908163,0.906145
max,0.953704,0.897727,0.918367,0.916456,0.897727,0.953704,0.918367,0.916456,0.897727,0.953704,0.918367,0.916456


In [30]:
rf = RandomForestClassifier(n_estimators=1500, max_features=.2, max_depth = 70, min_samples_split = 2, min_samples_leaf = 1, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 52.2 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.953241,0.896023,0.927551,0.924632,0.896023,0.953241,0.927551,0.924632,0.896023,0.953241,0.927551,0.924632
std,0.032698,0.032199,0.016185,0.015873,0.032199,0.032698,0.016185,0.015873,0.032199,0.032698,0.016185,0.015873
min,0.916667,0.852273,0.908163,0.906145,0.852273,0.916667,0.908163,0.906145,0.852273,0.916667,0.908163,0.906145
25%,0.925926,0.875000,0.911990,0.907513,0.875000,0.925926,0.911990,0.907513,0.875000,0.925926,0.911990,0.907513
50%,0.944444,0.886364,0.928571,0.926768,0.886364,0.944444,0.928571,0.926768,0.886364,0.944444,0.928571,0.926768
75%,0.979167,0.909091,0.940051,0.938184,0.909091,0.979167,0.940051,0.938184,0.909091,0.979167,0.940051,0.938184
max,1.000000,0.954545,0.954082,0.948864,0.954545,1.000000,0.954082,0.948864,0.954545,1.000000,0.954082,0.948864
